In [28]:
from pymongo import MongoClient
from sshtunnel import SSHTunnelForwarder
import bson

class MongoConnectionHandlerException(Exception):
    pass

class MongoConnectionHandler:

    __localhost = '127.0.0.1'

    def __init__(self):

        server = SSHTunnelForwarder(
            ('18.130.129.216', 22),
            ssh_username='mongo',
            ssh_password='7YPbhiYTEtM=',
            remote_bind_address=('192.168.2.91', 27017)
        )
        server.start()

        self.__client = MongoClient(host=self.__localhost, 
                                    port=server.local_bind_port, 
                                    username='mongo-admin', 
                                    password='7YPbhiYTEtM=', 
                                    retryWrites=False)

    @property
    def client(self):
        if self.__client is None:
            raise MongoConnectionHandlerException('Invalid database connection.')

        return self.__client

    def close(self):
        self.__client.close()

class MongoRepository:
    
    def __init__(self, client):
        
        self.__client = client
        
    def find_from_date(self, date):
        try:
            res = self.__client['dev_dexter_fuzzy_inference_recommendations']['recommendations_test_facebook'].find()
            res = list(res)
            return res
        except Exception as e:
            raise e
            
    def find_recommendation_instances(self):
        recs = {}
        res = self.__client['dev_recommendations']['recommendations_mj'].find()
        res = list(res)
#         print(res)
        for item in res:
            key = (item['structure_id'], item['metrics'][0]['name'])
            if key not in recs.keys():
                recs[key] = [item['template']]
            else:
                recs[key].append(item['template'])
                
        print(recs)
    
    def get_structure_details(self, structure_id):
        res = self.__client['dev_facebook_turing_structures']['ad'].find({'status': 1})
        res = list(res)
        decoded_stuff = [bson.BSON(res1['details']).decode() for res1 in res]
        return decoded_stuff
    
    def get_structures_by_account_id(self, account_id):
        import json
        res = self.__client['dev_facebook_turing_structures']['campaign'].find({'status': 1, 'account_id': account_id})
        res = list(res)
        decoded_stuff = [bson.BSON(res1['details']).decode() for res1 in res]
        ad_structures_file = open('campaign_structures.json', 'w+', encoding='utf-8')
        final_json = {}
        for i in range(0, len(res)):
            final_json[i] = decoded_stuff[i]
            
        ad_structures_file.write(json.dumps(final_json, indent=4))
        
mongo_client = MongoConnectionHandler()
mongo_repo = MongoRepository(mongo_client.client)

In [29]:
# altceva = mongo_repo.get_structure_details("6126161894188")
# print([ceva['name'] for ceva in altceva['targeting']['flexible_spec'][0]['interests']])

from dataclasses import dataclass

@dataclass
class Recommendation:
    action_breakdown = None
    ad_account_id = None
    breakdown = None
    campaign_id = None
    campaign_name = None
    category = None
    channel = None
    confidence = None
    created_at = None
    importance = None
    last_updated_at = None
    level = None
    metrics = None
    optimization_type = None
    parent_id = None
    parent_name = None
    recommendation_id = None
    redirect_for_edit = None
    source = None
    status = None
    structure_id = None
    structure_name = None
    template = None

In [30]:
    
import csv
mongo_client = MongoConnectionHandler()
mongo_repo = MongoRepository(mongo_client.client)
# result = mongo_repo.get_structure_details('23845029695300630')
# for res in result:
#     if 'recommendations' in res:
#         print(res)
#         break
recommendation_raw = mongo_repo.find_from_date('2020-06-02T21:20:00')

# mongo_repo.get_structures_by_account_id('169352866534012')

rec = [
    'Ad Account Id',
    'Breakdown',
    'Campaign Name',
    'Campaign ID',
    'Category',
    'Created at',
    'Confidence',
    'Importance',
    'Level',
    'Metrics',
    'Parent Id',
    'Parent name',
    'Recommendation type',
    'Source',
    'Structure Name',
    'Template',
    'Status',
    'Time interval',
]
with open('recommendations.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(rec)
    
    for recommendation in recommendation_raw:
        rec = []
        rec.append(recommendation['ad_account_id'])
        rec.append(recommendation['breakdown']['display_name'])
        rec.append(recommendation['campaign_name'])
        rec.append(recommendation['campaign_id'])
        rec.append(recommendation['category'])
        rec.append(recommendation['created_at'])
        rec.append(recommendation['confidence'])
        rec.append(recommendation['importance'])
        rec.append(recommendation['level'])
        try:
            rec.append([recommendation1['display_name'] for recommendation1 in recommendation['metrics']])
        except:
            rec.append("None")
        rec.append(recommendation['parent_id'])
        rec.append(recommendation['parent_name'])
        rec.append(recommendation['recommendation_type'])
        rec.append(recommendation['source'])
        rec.append(recommendation['structure_name'])
        rec.append(recommendation['template'])
        rec.append(recommendation['status'])
        rec.append(recommendation['time_interval']['value'])
        writer.writerow(rec)




